# Android WeatherApp Project Guide for absolute beginners - 3. MainActivity.java

I created this guide as a reference for my personal learning on Android app development. The original tutorial available in link in the references below.


**References**  
- GeeksforGeeks (2021) Making Weather App in Android Studio | Android Projects | GeeksforGeeks https://www.youtube.com/watch?v=q7NF-2gtfEU&t=864s

In [ ]:
from google.colab import drive
drive.mount('here')

In [ ]:
import cv2 
from google.colab.patches import cv2_imshow

def show_img(path):
    img = cv2.imread(path)
    cv2_imshow(img)

# MainActivity.java





## 1) Declare API variables


## 2) Initialise them in `onCreate`
inside the **`protected void onCreate(Bundle savedInstanceState){}`** method

### `setAdapter()`

- Excerpt from StackOverFlow:


    mDrawerList.setAdapter(new ArrayAdapter<String>
    (this,R.layout.drawer_list_item, mServices));


A list adapter is an object that adapts a collection objects for display in a ListView. ArrayAdapter is one simple implementation that maps an array of objects.

This line is mapping an array of strings (mServices) for display in a ListView (mDrawerList). The second argument to the adapter's constructor is the layout that will be used to render each list item.

- https://stackoverflow.com/questions/33703548/how-to-use-setadapter

In [ ]:
# MainActivity.java

public class MainActivity extends AppCompatActivity {

    # Declare variables with there id's.
    private RelativeLayout homeRL;

    ...

    private ImageView backIV, iconIV, searchIV;

    # Add WeatherRVModal variable here.
    # then initialise these down below in onCreate too.
    private ArrayList<WeatherRVModal> weatherRVModalArrayList;
    private WeatherRVAdapter weatherRVAdapter;  # adaptor class

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);

        # initialise the variables here.
        homeRL = findViewById(R.id.idRLHome);

        ...

        searchIV = findViewById(R.id.idIVSearch);
            
        # API we will going to use.
        weatherRVModalArrayList = new ArrayList<>();
        # Init ialise the adapter.
        weatherRVAdapter = new WeatherRVAdapter(this, weatherRVModalArrayList);
        # Set this adapter to the RecyclerView.
        weatherRV.setAdapter(weatherRVAdapter);

        # Inisialise the LocationManager.
        locationManager = (LocationManager) getSystemService(Context.LOCATION_SERVICE);
    }

...

## 3) Get weather info

In [ ]:
...

    # Create a method for weather info,
    # and another for user location.
    private  void getWeatherInfo(String cityName){

        # Parse the data from API.
        # so first, create a url.
        String url = "http://api.weatherapi.com/v1/forecast.json?key=34a7fe8762854dd7b8765811212509&q=" +cityName+ "&days=1&aqi=yes&alerts=yes";

    }

}

## 4) Get User Location

### 4-1) Configure Internet Permission in `AndroidManifest.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.weatherapp" >

    # Configure Internet Permission to use the user's current location..
    <uses-permission android:name="android.permission.INTERNET"/>
    <uses-permission android:name="android.permission.ACCESS_NETWORK_STATE"/>
    <uses-permission android:name="android.permission.ACCESS_COARSE_LOCATION"/>
    <uses-permission android:name="android.permission.ACCESS_FINE_LOCATION"/>

    <application
    
    ...

</manifest>    

### 4-2) `LocationManager` & `PERMISSION_CODE`

In [ ]:
public class MainActivity extends AppCompatActivity {

    ...

    # User location-related variables
    # We need to ask for permission first when using user's current location.
    private LocationManager locationManager;
    private int PERMISSION_CODE = 1;

    ...

### 4-3) Initialise `LocationManager`

- 1) Inisialise the LocationManager.
- 2) Check whether the user has granted permission or not. If the permission is not granted, ask the user to grant the permission.

In [ ]:
    ...

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);

        ...

        # 1) Inisialise the LocationManager.
        locationManager = (LocationManager) getSystemService(Context.LOCATION_SERVICE);
        # 2) Permission check=
        if(ActivityCompat.checkSelfPermission(this, Manifest.permission.ACCESS_FINE_LOCATION)!= PackageManager.PERMISSION_GRANTED && ActivityCompat.checkSelfPermission(this,Manifest.permission.ACCESS_COARSE_LOCATION)!=PackageManager.PERMISSION_GRANTED) {
            ActivityCompat.requestPermissions(MainActivity.this, new String[]{Manifest.permission.ACCESS_FINE_LOCATION, Manifest.permission.ACCESS_COARSE_LOCATION}, PERMISSION_CODE);
        }

        # Once the permission is granted,
        Location location = locationManager.getLastKnownLocation(LocationManager.NETWORK_PROVIDER);

    }

    ...

### 4-4) City name `getter`

1) Create a method that gets the city name from the API latitude and longitude data.  
2) Initialise with "Not found" in case the method fails to get longitude & latitude values.

In [ ]:
    ...

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);

    ...

    # 1) Create a method
    private String getCityName(double longitude, double latitude){
        # 2) Initialise CityName.
        String cityName = "Not found";
        Geocoder gcd = new Geocoder(getBaseContext(), Locale.getDefault());
        try {
            List<Address> addresses = gcd.getFromLocation(latitude, longitude, 10);

            for (Address adr: addresses){
                if (adr!=null){                           # if the locality data is there,
                    String city = adr.getLocality();      # get that data.
                    if (city!=null && !city.equals("")){  # check if the locality data is correct.
                        cityName = city;                  # update cityName with that screened data.
                    } else {
                        Log.d("TAG", "CITY NOT FOUND");
                        Toast.makeText(this, "User City Not Found", Toast.LENGTH_SHORT).show();
                    }
                }
            }
        } catch (IOException e){
            e.printStackTrace();
        }

        return cityName;

    }

    ...